In [220]:
import os

import numpy as np

import folium
from folium import plugins

print(folium.__version__)

0.11.0


In [221]:
import pandas as pd

In [222]:
df_covid = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

In [223]:
df_covid.head()

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN


In [224]:
df_countries_coord = pd.read_csv("https://gist.githubusercontent.com/tadast/8827699/raw/3cd639fa34eec5067080a61c69e3ae25e3076abb/countries_codes_and_coordinates.csv")

In [225]:
for col in df_countries_coord.columns:
    df_countries_coord[col] = df_countries_coord[col].apply(lambda x:x.replace('"',''))

In [226]:
df_countries_coord['Alpha-3 code'] = df_countries_coord['Alpha-3 code'].apply(lambda x:x.strip())

In [227]:
df_countries_coord['Latitude (average)'] = df_countries_coord['Latitude (average)'].astype(float)

In [228]:
df_countries_coord['Longitude (average)'] = df_countries_coord['Longitude (average)'].astype(float)

In [229]:
df_countries_coord.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6


In [230]:
df_merged = pd.merge(df_covid, df_countries_coord, left_on='iso_code', right_on='Alpha-3 code', how='inner')

In [231]:
df_merged = df_merged[['date','new_deaths','Latitude (average)','Longitude (average)']]

In [232]:
df_merged["w_lat"] = df_merged["new_deaths"] * df_merged["Latitude (average)"]

In [233]:
df_merged["w_lon"] = df_merged["new_deaths"] * df_merged["Longitude (average)"]

In [234]:
df_merged.head()

,date,new_deaths,Latitude (average),Longitude (average),w_lat,w_lon
0,2020-03-13,0,12.5,-69.9667,0.0,-0.0
1,2020-03-20,0,12.5,-69.9667,0.0,-0.0
2,2020-03-24,0,12.5,-69.9667,0.0,-0.0
3,2020-03-25,0,12.5,-69.9667,0.0,-0.0
4,2020-03-26,0,12.5,-69.9667,0.0,-0.0


In [235]:
df_merged = df_merged.groupby('date').sum().reset_index()

In [236]:
df_merged = df_merged.loc[df_merged['new_deaths'] > 0]

In [237]:
df_merged['w_center_lat'] = df_merged['w_lat'] / df_merged['new_deaths']
df_merged['w_center_lon'] = df_merged['w_lon'] / df_merged['new_deaths']

In [238]:
df_merged = df_merged[['date','w_center_lat','w_center_lon']]

In [239]:
df_merged['w_center_lat'] = df_merged['w_center_lat'].rolling(10).mean()

In [240]:
df_merged['w_center_lon'] = df_merged['w_center_lon'].rolling(10).mean()

In [241]:
df_merged = df_merged.iloc[9:]

In [242]:
dict_list = df_merged.to_dict(orient='records')

In [243]:
dict_list

[{'date': '2020-01-28', 'w_center_lat': 35.0, 'w_center_lon': 105.0},
 {'date': '2020-01-29', 'w_center_lat': 35.0, 'w_center_lon': 105.0},
 {'date': '2020-01-30', 'w_center_lat': 35.0, 'w_center_lon': 105.0},
 {'date': '2020-01-31', 'w_center_lat': 35.0, 'w_center_lon': 105.0},
 {'date': '2020-02-01', 'w_center_lat': 35.0, 'w_center_lon': 105.0},
 {'date': '2020-02-02',
  'w_center_lat': 34.952173913043474,
  'w_center_lon': 105.03695652173913},
 {'date': '2020-02-03',
  'w_center_lat': 34.952173913043474,
  'w_center_lon': 105.03695652173913},
 {'date': '2020-02-04',
  'w_center_lat': 34.952173913043474,
  'w_center_lon': 105.03695652173913},
 {'date': '2020-02-05',
  'w_center_lat': 34.952173913043474,
  'w_center_lon': 105.03695652173913},
 {'date': '2020-02-06',
  'w_center_lat': 34.952173913043474,
  'w_center_lon': 105.03695652173913},
 {'date': '2020-02-07',
  'w_center_lat': 34.952173913043474,
  'w_center_lon': 105.03695652173913},
 {'date': '2020-02-08',
  'w_center_lat': 34

In [244]:
l_coords = []

for i in range(len(dict_list) - 1):
    d = {}
    p1 = dict_list[i]
    p2 = dict_list[i+1]
    d['coordinates'] = [
        [p1['w_center_lon'],p1['w_center_lat']],
        [p2['w_center_lon'],p2['w_center_lat']]
    ]
    d['dates'] = [
        p1['date'],
        p2['date']
    ]
    d['color'] = 'red'
    l_coords.append(d)

In [245]:
l_coords

[{'coordinates': [[105.0, 35.0], [105.0, 35.0]],
  'dates': ['2020-01-28', '2020-01-29'],
  'color': 'red'},
 {'coordinates': [[105.0, 35.0], [105.0, 35.0]],
  'dates': ['2020-01-29', '2020-01-30'],
  'color': 'red'},
 {'coordinates': [[105.0, 35.0], [105.0, 35.0]],
  'dates': ['2020-01-30', '2020-01-31'],
  'color': 'red'},
 {'coordinates': [[105.0, 35.0], [105.0, 35.0]],
  'dates': ['2020-01-31', '2020-02-01'],
  'color': 'red'},
 {'coordinates': [[105.0, 35.0], [105.03695652173913, 34.952173913043474]],
  'dates': ['2020-02-01', '2020-02-02'],
  'color': 'red'},
 {'coordinates': [[105.03695652173913, 34.952173913043474],
   [105.03695652173913, 34.952173913043474]],
  'dates': ['2020-02-02', '2020-02-03'],
  'color': 'red'},
 {'coordinates': [[105.03695652173913, 34.952173913043474],
   [105.03695652173913, 34.952173913043474]],
  'dates': ['2020-02-03', '2020-02-04'],
  'color': 'red'},
 {'coordinates': [[105.03695652173913, 34.952173913043474],
   [105.03695652173913, 34.952173913

In [246]:
m = folium.Map(
    location=[35.68159659061569, 139.76451516151428],
    zoom_start=16
)

# Lon, Lat order.
lines = l_coords

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='P1D', add_last_point=False).add_to(m)

m.save('Plugins_6.html')
m